# 🔍 Fun with Keyword Search using VideoDB -2

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/experiment-keyword-search-2/examples/Keyword_Search_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
---

### 📦  Installing packages 

In [ ]:
%pip install videodb

### 🔑 API Keys

In [3]:
import os

os.environ["VIDEO_DB_API_KEY"] = ""

## Steps
---

### 🌐 Step 1: Connect to VideoDB

In [4]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Videos


In [5]:
video = coll.upload(url="https://www.youtube.com/watch?v=XEzRZ35urlk")
video.play()

### 🔊  Step 3: Index Spoken Words


In [6]:
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.52it/s]


### 🔎  Step 4: Search for any keyword

In [7]:
from videodb import SearchType

queries = [" ai", " generative", " agents"]
results = []

for query in queries:
    results.append(video.search(query, search_type=SearchType.keyword))

In [8]:
shots = []
padding = 0

for result in results:
    for shot in result.get_shots():
        shots.append([shot.start - padding, shot.end + padding])

In [9]:
def merge_intervals(intervals):
    # Step 1: Sort intervals based on the start time
    intervals.sort(key=lambda x: x[0])

    # Step 2: Initialize the merged intervals list
    merged_intervals = []

    for interval in intervals:
        # If the merged intervals list is empty or there is no overlap
        if not merged_intervals or merged_intervals[-1][1] < interval[0]:
            merged_intervals.append(interval)
        else:
            # There is an overlap, so merge the current interval with the last one
            merged_intervals[-1][1] = max(merged_intervals[-1][1], interval[1])

    return merged_intervals

In [10]:
merged_shots = merge_intervals(shots)

In [11]:
from videodb.timeline import Timeline 
from videodb.asset import VideoAsset, TextAsset

timeline = Timeline(conn)
seeker = 0
for shot in merged_shots:
    duration = shot[-1] - shot[0]

    video_asset = VideoAsset(
        asset_id=video.id, start=shot[0], end=shot[-1]
    )

    timeline.add_inline(asset=video_asset)

    seeker += duration

In [12]:
from videodb import play_stream

stream_url = timeline.generate_stream()
print(stream_url)
play_stream(stream_url)

https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/1caaf1a9-8fbe-44f4-b2a8-51741b3783fd.m3u8


'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/1caaf1a9-8fbe-44f4-b2a8-51741b3783fd.m3u8'